# **Deploy ML model to AKS webservice, including 6 steps as follows**

## **Step1: get the workspace, source directory and environment(with conda dependencies)**

In [ ]:
from azureml.core import Workspace, Environment
from azureml.core.conda_dependencies import CondaDependencies


ws = Workspace.from_config()
source_dir = "<YOUR_SOURCE_DIR>"#eg.'/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci-mlopspoc/code/Users/poc/'
env = Environment(name="<YOUR_ENV_NAME>",version="<TARGET_VERSION>")

# keep service principal password into env variables
env.environment_variables= {'AZUREML_PASSWORD': '<YOUR_SERVICE_PRINCIPAL_PASSWORD>'}

#Optional:
#env.environment_variables = {'AZUREML_MODEL_DIR': 'model_env'}

# conda denpendencies configuration
conda_dep = CondaDependencies()
conda_dep.add_pip_package("pillow")
conda_dep.add_pip_package("torch == 1.4.0")
conda_dep.add_pip_package("torchvision == 0.5.0" )
conda_dep.add_pip_package("pandas" )
conda_dep.add_pip_package("'azureml-monitoring'" )
env.python.conda_dependencies=conda_dep
with open('myenv.yml', 'w') as f:
    f.write(env._serialize_to_string())


## **Step2: configuring inference object**

In [17]:
from azureml.core.model import InferenceConfig, Model

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = source_dir,
    entry_script = "prediction.py",
)

## **Step3: get the model**

In [ ]:
model = Model(ws,'<YOUR_MODEL_NAME>',version="<YOUR_MODEL_VERSION>")

## **Step4: configuring AKS compute target and AKS deployment configuration**

In [18]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,"<Your AKS Cluster Name>")
deployment_config = AksWebservice.deploy_configuration()

## **Step5: deploy to AKS webservice**

In [ ]:
service = Model.deploy(
    ws,
    "<YOUR-SERVICE-NAME>",
    [model],
    dummy_inference_config,
    deployment_config,
    aks_target,
    overwrite = True,
)
# or another way
#service = Webservice.deploy_from_model(ws, 'testservice-aks', deployment_config, [model], dummy_inference_config)
service.wait_for_deployment(show_output=True)#Start up a container in AKS using the image
print(service.get_logs())

## **Step6: get the REST API(URI) of this service**

In [ ]:
uri = service.scoring_uri
print(uri)

# **Call the service deployed on AKS with REST API via python SDK**

In [23]:
# call the model using python SDK
import requests

uri = '<YOUR_SERVER_URI>'
api_key = '<YOUR_API_KEY>'
headers = {'Authorization':('Bearer '+ api_key)}

image_path = '<YOUR_IMAGE_PATH>'
files = {'image': open(image_path, 'rb').read()}

response = requests.post(uri, files = files, headers = headers)
if response.ok:
    print(response.json())

{'pred_class': {'0': 'pass'}}


## **Data collection is not automatically set to true when you deploy a service in AKS. Update your configuration file. You can also enable Application Insights for service monitoring by changing this configuration as in the following example:**

In [ ]:
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)

## **### Disable data collection ### **
## replace <service_name> with the name of the web service
<service_name>.update(collect_model_data=False)